In [1]:
import numpy as np
from pathlib import Path
from collections import Counter
import os
import re
import nltk

os.chdir('../')

from src.data import read_sentences

In [2]:
SEED = 0

In [209]:
def annotate_sentences(tagged_tr, d_senses, lemma=False, sel_verbs=True):
    tagged_sense_sent = []
    input_sense_sent = []
    orig_sent = []
    cont = 0
    a = 0
    for instance in tagged_tr:
        temp_o = []
        temp_i = []
        for ix, word in enumerate(instance[1].split()):
            if instance[0][ix] != 'no_instance':
                if len(wordnet_senses) != 0:
                    if sel_verbs and d_senses[instance[0][ix]][0] in wordnet_senses and (nlp_pt(word)[0].lemma_ in PT_DICT_VERBS or wordnet_senses[d_senses[instance[0][ix]][0]] in SELECTED_SYNSETS):
                        temp_o.append(wordnet_senses[d_senses[instance[0][ix]][0]])
                        if lemma:
                            d = nlp_pt(word)
                            lemma = d[0].lemma_ 
                            if lemma == 'deixar-me':
                                lemma = 'deixar'
                            temp_i.append(lemma + '_tag')
                        else:
                            temp_i.append(word)
                        a += 1
                    elif not sel_verbs and d_senses[instance[0][ix]][0] in wordnet_senses:
                        temp_o.append(wordnet_senses[d_senses[instance[0][ix]][0]])
                        if lemma:
                            d = nlp_pt(word)
                            lemma = d[0].lemma_ 
                            if lemma == 'deixar-me':
                                lemma = 'deixar'
                            temp_i.append(lemma + '_tag')
                        else:
                            temp_i.append(word)
                        a += 1
                    else:
                        cont += 1
                        temp_o.append(word)
                        temp_i.append(word)
                else:
                    temp_o.append(d_senses[instance[0][ix]][0])
                    temp_i.append(word)
            else:
                temp_o.append(word)
                temp_i.append(word)

        input_sense_sent.append(' '.join(temp_i))
        tagged_sense_sent.append(' '.join(temp_o))
        orig_sent.append(instance[2])
    
    return tagged_sense_sent, input_sense_sent, orig_sent

def save_tagged_sentence(dir_path, name_file, tagged_sense_sent, input_sense_sent, orig_sent, mode='w'):
    
    with open(dir_path / f'{name_file}_out.txt', mode) as file:
        for sent in tagged_sense_sent:
            file.write(sent + '\n')
    file.close()
    
    with open(dir_path / f'{name_file}_in.txt', mode) as file:
        for sent in input_sense_sent:
            file.write(sent + '\n')
    file.close() 
    
    with open(dir_path / f'{name_file}_orig.txt', mode) as file:
        for sent in orig_sent:
            file.write(sent + '\n')
    file.close()     
    
    

In [4]:
PT_VERBS = ['tratar', 'estabelecer', 'marcar', 'vir', 'colocar',\
           'fechar', 'dar', 'cair', 'encontrar', 'registrar',\
           'levar', 'receber', 'apresentar', 'passar', 'deixar',\
           'chegar', 'ficar', 'fazer', 'ter', 'ser']

In [5]:
SELECTED_SYNSETS = np.load(Path.cwd() / 'data/disambiguation/selected_synsets.npy')

In [6]:
PT_DICT_VERBS ={
    'tratar': ['deal with', 'treat', 'process', 'deal', 'manage', 'do', 'attend', 'look after', 'cherrish', 'misuse', 'size'],\
    'estabelecer': ['establish', 'set', 'fix', 'lay down', 'make', 'settle', 'determine', 'presribe', 'impose', 'enter into', 'stipulate', 'organize', 'seat'],\
    'marcar': ['mark', 'brand', 'dial', 'book', 'stamp', 'show', 'read', 'define', 'trace', 'earmark', 'reserve', 'feature', 'signilize', 'scribe'],\
    'vir': ['come', 'arrive', 'come on', 'come up with'],\
    'colocar': ['put', 'lay', 'set', 'place', 'post', 'pose', 'stick', 'plant', 'dispose', 'collocate', 'posit', 'bestow', 'pitch', 'clap', 'ship', 'placatory'],\
    'fechar': ['close', 'shut', 'seal', 'turn off', 'pin down', 'fasten', 'occlude', 'shut in', 'box', 'impount', 'bar', 'berate', 'stop up', 'mure', 'stopple', 'rail', 'pen', 'inmure'],\
    'dar': ['to give', 'give', 'impart', 'provide', 'render', 'afford', 'yield', 'allow', 'hand', 'deal', 'administer', 'give in', 'gift', 'confer', 'inflict', 'handsel', 'accord'],\
    'cair': ['fall', 'go down', 'drop', 'sink', 'collapse', 'founder', 'topple', 'lapse', 'sleet', 'keel over', 'prey', 'prostrate', 'pelt', 'plump', 'flump'],\
    'encontrar': ['meet', 'find', 'detect', 'encounter', 'find out', 'discover', 'meet with', 'experience', 'get together', 'impinge', 'hunt up'],\
    'registrar': ['register', 'record', 'read', 'book', 'enroll', 'inscribe', 'enrol', 'list', 'write down', 'set down', 'trace', 'score', 'label', 'matriculate', 'prick down', 'calendar', 'signalize'],\
    'levar': ['take along', 'take', 'carry', 'convey', 'go', 'prompt', 'induce', 'hold', 'charge', 'ravish'],\
    'receber': ['to receive', 'receive', 'welcome', 'get', 'have', 'accept', 'collect', 'meet', 'entertain', 'do', 'reap', 'derive', 'salute'],\
    'apresentar': ['to present', 'present', 'introduce', 'show', 'exhibit', 'lodge', 'produce', 'put', 'bring forward', 'come up with', 'represent', 'bring up', 'render'],\
    'passar': ['pass', 'spend', 'hand', 'go', 'go by', 'elapse', 'slip away', 'come', 'transfer', 'give in'],\
    'deixar': ['leave', 'let', 'have', 'quit', 'let go', 'depart', 'go away', 'drop off', 'leave out'],\
    'chegar': ['to arrive', 'arrive', ' get in', 'come', 'achieve', 'land', 'get around', 'turn up'],\
    'ficar': ['stay', 'bide', 'be', 'remain', 'go', 'continue', 'keep', 'come'],\
    'fazer': ['do', 'perform', 'make', 'cause', 'create', 'produce', 'render', 'manufacture'],\
    'ter': ['tue', 'have', 'take'],\
    'ser': ['to be', 'be', 'being']
}

# Eval senses tagged

## Read translations

In [19]:
or_file_semcor = open('data/translation/semcor/en_pt/semcor_or_2.txt', 'r')
tr_file_semcor = open('data/translation/semcor/en_pt/semcor_tr_2.txt', 'r')
id_file_semcor = open('data/translation/semcor/en_pt/semcor_id_2.txt', 'r')
al_file_semcor = open('data/translation/semcor/en_pt/semcor_al_2.txt', 'r')

or_file_omsti = open('data/translation/omsti/en_pt/omsti_or.txt', 'r')
tr_file_omsti = open('data/translation/omsti/en_pt/omsti_tr.txt', 'r')
id_file_omsti = open('data/translation/omsti/en_pt/omsti_id.txt', 'r')
al_file_omsti = open('data/translation/omsti/en_pt/omsti_al.txt', 'r')

or_file_dev = open('data/translation/dev/en_pt/dev_or.txt', 'r')
tr_file_dev = open('data/translation/dev/en_pt/dev_tr.txt', 'r')
id_file_dev = open('data/translation/dev/en_pt/dev_id.txt', 'r')
al_file_dev = open('data/translation/dev/en_pt/dev_al.txt', 'r')

In [11]:
def parse_alignment(chars):
    alignment = []
    for e in re.split(r'[,|;]', chars):
        temp = e.split('-')
        range_a = temp[0].split(':')
        range_b = temp[1].split(':')
        alignment.append([np.arange(int(range_a[0]), int(range_a[0]) + int(range_a[1]) + 1),\
                         np.arange(int(range_b[0]), int(range_b[0]) + int(range_b[1]) + 1)])
        
    return alignment

In [12]:
def load_arrays(or_file, tr_file, id_file, al_file):
    tr_ar = []
    for sent in tr_file:
        tr_ar.append(sent.replace('\n', ''))
        
    or_ar = []
    for sent in or_file:
        or_ar.append(sent.replace('\n', ''))

    id_ar = []
    for sent in id_file:
        id_ar.append(int(sent.replace('\n', '')))

    al_ar = []
    for sent in al_file:
        al_ar.append(parse_alignment(sent.replace('\n', '')))
        
    return np.array(or_ar), np.array(tr_ar), np.array(id_ar), np.array(al_ar)

In [20]:
or_semcor, tr_semcor, id_semcor, al_semcor = load_arrays(or_file_semcor, tr_file_semcor, id_file_semcor, al_file_semcor)
or_omsti, tr_omsti, id_omsti, al_omsti = load_arrays(or_file_omsti, tr_file_omsti, id_file_omsti, al_file_omsti)
or_dev, tr_dev, id_dev, al_dev = load_arrays(or_file_dev, tr_file_dev, id_file_dev, al_file_dev)

In [54]:
or_all = np.concatenate((or_semcor, or_omsti, or_dev), axis=0)
tr_all = np.concatenate((tr_semcor, tr_omsti, tr_dev), axis=0)
id_all = np.concatenate((id_semcor, id_omsti, id_dev), axis=0)
al_all = np.concatenate((al_semcor, al_omsti, al_dev), axis=0)

In [21]:
len(or_semcor), len(tr_semcor), len(id_semcor), len(al_semcor)

(37163, 37163, 37163, 37163)

In [22]:
len(or_omsti), len(tr_omsti), len(id_omsti), len(al_omsti)

(40260, 40260, 40260, 40260)

In [23]:
len(or_dev), len(tr_dev), len(id_dev), len(al_dev)

(145, 145, 145, 145)

In [55]:
len(or_all), len(tr_all), len(id_all), len(al_all)

(77568, 77568, 77568, 77568)

In [24]:
or_file_semcor.close()
tr_file_semcor.close()
id_file_semcor.close()
al_file_semcor.close()

or_file_omsti.close()
tr_file_omsti.close()
id_file_omsti.close()
al_file_omsti.close()

or_file_dev.close()
tr_file_dev.close()
id_file_dev.close()
al_file_dev.close()

## Read corpus annotations

In [25]:
input_sent_semcor, sent_to_id_semcor, target_sent_semcor = read_sentences(Path.cwd() / 'data/WSD_Evaluation_Framework/Training_Corpora/', 'SemCor', False)
input_sent_dev, sent_to_id_dev, target_sent_dev = read_sentences(Path.cwd() / 'data/WSD_Evaluation_Framework/Evaluation_Datasets/', 'ALL', False)
input_sent_temp, sent_to_id_temp, target_sent_temp = read_sentences(Path.cwd() / 'data/WSD_Evaluation_Framework/Training_Corpora/', 'SemCor+OMSTI', True)

# Filter sentences
input_sent_omsti = []
sent_to_id_omsti = []
target_sent_omsti = []
for ix, sent in enumerate(input_sent_temp):
    if sent not in input_sent_semcor:
        input_sent_omsti.append(sent)
        sent_to_id_omsti.append(sent_to_id_temp[ix])
        target_sent_omsti.append(target_sent_temp[ix])

## Check amount of valid data

In [26]:
def get_alignment(pos_word, al):
    temp = []
    for e in al:
        if pos_word in e[1]:
            temp.append(e)
    
    if len(temp):
        temp = sorted(temp, key= lambda x: len(x[1]))
        temp = temp[0]
        
    return temp

In [27]:
def count_words_with_verb(sent):
    temp_ix = []
    temp_cont = []
    cont_char = 0
    for ix, word in enumerate(sent.split()):
        if word in PT_VERBS:
            temp_cont.append(cont_char)
            temp_ix.append(ix)
        cont_char += len(word) + 1
    return temp_cont, temp_ix

In [28]:
def count_tagged_instance(id_arr, al_arr, tr_arr, input_sent_arr, sent_to_id_arr, target_sent_arr):
    cont = 0
    for ix_tr, ix_input_sent in enumerate(id_arr):
        
        input_sent = input_sent_arr[ix_input_sent]
        sent_to_id = sent_to_id_arr[ix_input_sent]
        target_sent = target_sent_arr[ix_input_sent]
        
        al = al_arr[ix_tr]
        tr = tr_arr[ix_tr]
        
        arr_cont_char, arr_pos_word = count_words_with_verb(tr)
        if len(arr_cont_char):
            for cont_char, pos_word in zip(arr_cont_char, arr_pos_word):
                alingment = get_alignment(cont_char, al)
                #print(alingment)
                t_or = min(alingment[0])
                d_or = max(alingment[0])
                t_tr = min(alingment[1])
                d_tr = max(alingment[1])

                ant_n_words = len(input_sent[:t_or].split())
                n_words = len(input_sent[t_or:d_or].split())
                for id_sense in sent_to_id[ant_n_words:(ant_n_words+n_words)]:
                    if target_sent[id_sense] != 'no_instance':
                        cont +=1
    
    return cont

In [30]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(' ')
        # All tokens 'own' a subsequent space character in this tokenizer
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)


In [31]:
import spacy

nlp_pt = spacy.load('pt_core_news_sm')
nlp_pt.tokenizer = WhitespaceTokenizer(nlp_pt.vocab)

nlp_en = spacy.load('en_core_web_sm')
nlp_en.tokenizer = WhitespaceTokenizer(nlp_en.vocab)

# Eval metrics

In [32]:
or_file_semcor = open('data/translation/semcor/pt_en/semcor_or_2.txt', 'r')
tr_file_semcor = open('data/translation/semcor/pt_en/semcor_tr_2.txt', 'r')
al_file_semcor = open('data/translation/semcor/pt_en/semcor_al_2.txt', 'r')

or_file_omsti = open('data/translation/omsti/pt_en/omsti_or_2.txt', 'r')
tr_file_omsti = open('data/translation/omsti/pt_en/omsti_tr_2.txt', 'r')
al_file_omsti = open('data/translation/omsti/pt_en/omsti_al_2.txt', 'r')

In [39]:
or_file_dev = open('data/translation/dev/pt_en/dev_or.txt', 'r')
tr_file_dev = open('data/translation/dev/pt_en/dev_tr.txt', 'r')
al_file_dev = open('data/translation/dev/pt_en/dev_al.txt', 'r')

In [34]:
bck_or_semcor, bck_tr_semcor, _, bck_al_semcor = load_arrays(or_file_semcor, tr_file_semcor, [], al_file_semcor)
bck_or_omsti, bck_tr_omsti, _, bck_al_omsti = load_arrays(or_file_omsti, tr_file_omsti, [], al_file_omsti)
bck_or_dev, bck_tr_dev, _, bck_al_dev = load_arrays(or_file_dev, tr_file_dev, [], al_file_dev)

In [56]:
bck_or_all = np.concatenate((bck_or_semcor, bck_or_omsti, bck_or_dev), axis=0)
bck_tr_all = np.concatenate((bck_tr_semcor, bck_tr_omsti, bck_tr_dev), axis=0)
bck_al_all = np.concatenate((bck_al_semcor, bck_al_omsti, bck_al_dev), axis=0)

In [387]:
or_all[0]

'how long has it been since you reviewed the objectives of your benefit and service program ?'

In [41]:
or_file_semcor.close()
tr_file_semcor.close()
al_file_semcor.close()

or_file_omsti.close()
tr_file_omsti.close()
al_file_omsti.close()

or_file_dev.close()
tr_file_dev.close()
al_file_dev.close()

In [42]:
bck_tr_semcor[-100], or_semcor[-100]

('the next days may show where things are .',
 'the next days may show where things stand .')

In [43]:
bck_tr_omsti[-100], or_omsti[-100]

('whether as athletes or spectators , when we find ourselves in and participate in international sporting events , we share the brilliance of world-class competition , whether it be in the pain of losing, or the glory of winning , but more importantly , on a willingness of participation .',
 'whether as athletes or spectators , when we meet at and participate in international sporting events , we share in the glow of world-class competition , whether it be in the heartbreak of losing or in the glory of winning , but most importantly , in the goodwill of participation .')

In [44]:
bck_tr_dev[-100], or_dev[-100]

('it is clear that on Thursday , haney mailed the monthly check for separate maintenance to his wife lolly , and wished the stranger could do something about it',
 'of course on thursday , haney mailed the monthly check for separate maintenance to his wife lolly , and wished the stranger could do something about her')

## Eval Rouge

In [45]:
import rouge

In [46]:
def get_rouge_per_sentence(input_sentences, trns_sentences):
    f1_scores = []
    for sent, trns in zip(input_sentences, trns_sentences):
        evaluator = rouge.Rouge(metrics=['rouge-n'],
                               max_n=2,
                               limit_length=True,
                               length_limit=100,
                               length_limit_type='words',
                               alpha=0.5, # Default F1_score
                               weight_factor=1.2,
                               stemming=True)

        all_hypothesis = [trns]
        all_references = [sent]

        scores = evaluator.get_scores(all_hypothesis, all_references)
        for metric, results in sorted(scores.items(), key=lambda x: x[0]):
            if metric == 'rouge-2':
                f1_scores.append(results['f'])

    return np.array(f1_scores)

In [57]:
rouge_all = get_rouge_per_sentence(or_all, bck_tr_all)

## Eval Bleu

In [58]:
import nltk
from nltk.translate.bleu_score import SmoothingFunction

def get_bleu_per_sentence(input_sentences, trns_sentences):
    bleu_scores = []
    smoothie = SmoothingFunction().method4
    for sent, trns in zip(input_sentences, trns_sentences):
        hypothesis = trns.split() 
        reference = sent.split()
        
        references = [reference]
        bleu = nltk.translate.bleu_score.sentence_bleu(references, hypothesis, smoothing_function=smoothie)
        
        bleu_scores.append(bleu)

    return np.array(bleu_scores)

In [59]:
bleu_all = get_bleu_per_sentence(or_all, bck_tr_all)

## Eval f1

In [64]:
def get_f1_per_sentence(bleu, rouge):
    
    f1 = []
    for b, r in zip(bleu, rouge):
        if b != 0 or r != 0:
            f1.append(2 * (b * r) / (b + r))
        else:
            f1.append(0)
        
    return np.array(f1)

In [65]:
f1_all = get_f1_per_sentence(bleu_all, rouge_all)

## Eval meteor

In [67]:
def get_meteor_per_sentence(name):
    
    meteor = []
    with open(f'data/meteor_output/{name}.txt') as f:
        for line in f.read().split('\n'):
            if 'Segment' in line and 'score' in line:
                meteor.append(float(line.split(':')[1]))
        
    return np.array(meteor)

In [184]:
meteor_semcor = get_meteor_per_sentence('semcor')
meteor_omsti = get_meteor_per_sentence('omsti')
meteor_dev = get_meteor_per_sentence('dev')
meteor_all = np.concatenate((meteor_semcor, meteor_omsti, meteor_dev), axis=0)

## Eval similarity

In [69]:
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
import re, string, unicodedata

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words


def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words


def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words


def normalize_text(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words


In [70]:
import html
import scipy 

def calc_vector(text, model):
    item = html.unescape(text)
    gen_doc = [w for w in word_tokenize(item)]
    normalized = normalize_text(gen_doc)
    
    model.random.seed(SEED)
    return model.infer_vector(normalized, epochs=6)

def similarity(text1, text2, model):
    vec1 = calc_vector(text1, model)
    vec2 = calc_vector(text2, model)    
    return 1-scipy.spatial.distance.cosine(vec1, vec2)

In [71]:
def get_sim_per_sentence(or_sents, bck_sents):
    
    sims = []
    for or_s, bck_s in zip(or_sents, bck_sents):
        sims.append(similarity(or_s, bck_s, model))
        
    return np.array(sims)

In [76]:
from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec.load('data/vectors/enwiki_doc2vec')

In [77]:
sim_semcor = get_sim_per_sentence(or_semcor, bck_tr_semcor)
sim_omsti = get_sim_per_sentence(or_omsti, bck_tr_omsti)
sim_dev = get_sim_per_sentence(or_dev, bck_tr_dev)

In [80]:
sim_all = np.concatenate((sim_semcor, sim_omsti, sim_dev))

In [183]:
len(f1_all)

77568

# Transform translations

In [82]:
#rouge_omsti = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'rouge.npy')[:len(id_omsti)]
#bleu_omsti = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'bleu.npy')[:len(id_omsti)]
#f1_omsti = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'f1.npy')[:len(id_omsti)]
#meteor_omsti = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'meteor.npy')[:len(id_omsti)]
#sim_omsti = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'sim.npy')[:len(id_omsti)]

#rouge_semcor = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'rouge.npy')[len(id_omsti):]
#bleu_semcor = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'bleu.npy')[len(id_omsti):]
#f1_semcor = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'f1.npy')[len(id_omsti):]
#meteor_semcor = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'meteor.npy')[len(id_omsti):]
#sim_semcor = np.load(Path.cwd() / 'data/disambiguation/all_less_mean' / 'sim.npy')[len(id_omsti):]

In [83]:
def count_words(sent):
    temp_ix = []
    temp_cont = []
    cont_char = 0
    for ix, word in enumerate(sent.split()):
        temp_cont.append(cont_char)
        temp_ix.append(ix)
        
        cont_char += len(word) + 1
        
    return temp_cont, temp_ix

In [84]:
def check_word(tr_word, orig_words):
    if tr_word in PT_DICT_VERBS:
        orig_options = PT_DICT_VERBS[tr_word]
        for orig_word in orig_words:
            if orig_word in orig_options:
                return True
                
    return False

In [85]:
def get_pos_tags(orig_sentences, trns_sentences): 
    tags_tr = []
    for sent in (trns_sentences):
        if len(sent):
            tag_tr = []
            temp = nlp_pt(str(sent).strip())
            for token in temp:
                tag_tr.append([token.text, token.pos_])
            tags_tr.append(tag_tr)
        else:
            tags_tr.append(['invalid', 'invalid'])
    
    tags_or = []
    for sent in (orig_sentences):
        if len(sent):
            tag_or = []
            temp = nlp_en(str(sent).strip())
            for token in temp:
                tag_or.append([token.text, token.pos_])
            tags_or.append(tag_or)
        else:
            tags_or.append(['invalid', 'invalid'])
        
    return tags_or, tags_tr

In [86]:
def tagged_translations(selected_id_tr, id_arr, al_arr, tr_arr, input_sent_arr, sent_to_id_arr, target_sent_arr, pos_or, pos_tr):
    cont = 0
    tagged_tr = []
    
    for ix_tr, ix_input_sent in zip(selected_id_tr, id_arr.take(selected_id_tr)):
        input_sent = input_sent_arr[ix_input_sent]
        sent_to_id = sent_to_id_arr[ix_input_sent]
        target_sent = target_sent_arr[ix_input_sent]
        
        al = al_arr[ix_tr]
        tr = tr_arr[ix_tr]
        
        arr_cont_char, arr_pos_word = count_words(tr)
        if len(arr_cont_char):
            
            tags_tr = pos_tr[ix_tr]
            tags_or = pos_or[ix_input_sent]
            if verbose:
                print()
                print('sentence orig: ', input_sent)
                print('sentence translated: ', tr)
            temp = []
            for cont_char, pos_word in zip(arr_cont_char, arr_pos_word):
                alingment = get_alignment(cont_char, al)
                if len(alingment):
                    t_or = min(alingment[0])
                    d_or = max(alingment[0])
                    t_tr = min(alingment[1])
                    d_tr = max(alingment[1])

                    ant_n_words_or = len(input_sent[:t_or].split())
                    n_words_or = len(input_sent[t_or:d_or].split())
                    
                    ant_n_words_tr = len(tr[:t_tr].split())
                    n_words_tr = len(tr[t_tr:d_tr].split())
                    
                    input_sent_tokens = input_sent.split()[ant_n_words_or:(ant_n_words_or+n_words_or)]
                    trns_sent_tokens = tr.split()[ant_n_words_tr:(ant_n_words_tr+n_words_tr)]
                    
                    if verbose:
                        print()
                        print('translated word: ', tr.split()[pos_word])
                        print('senses: ', np.array(target_sent).take(sent_to_id[ant_n_words_or:(ant_n_words_or+n_words_or)]))
                        print('token input original: ', input_sent_tokens)
                        print('token translations: ', tr.split()[ant_n_words_tr:(ant_n_words_tr+n_words_tr)])
                        print('pos original: ', tags_tr[pos_word])
                        print('pos translation: ', tags_or[ant_n_words_or:(ant_n_words_or+n_words_or)])
                    if len(input_sent_tokens) == 1:   
                        
                        tag_or = tags_or[ant_n_words_or:(ant_n_words_or+n_words_or)][0]
                        tag_tr = tags_tr[pos_word]
                        id_sense = sent_to_id[ant_n_words_or:(ant_n_words_or+n_words_or)][0]
                        if verbose:
                            print('token 1: ', target_sent[id_sense])
                        if target_sent[id_sense] != 'no_instance' and tag_or[1] == tag_tr[1]: 
                            temp.append(target_sent[id_sense])
                        else:
                            temp.append('no_instance') 

                    elif len(input_sent_tokens) <= 4 and len(trns_sent_tokens) <= 5 and check_word(tr.split()[pos_word], input_sent[t_or:d_or].split()):
                        is_ambiguous = False

                        for id_sense, word in zip(sent_to_id[ant_n_words_or:(ant_n_words_or+n_words_or)], input_sent_tokens):
                            if target_sent[id_sense] != 'no_instance' and word in PT_DICT_VERBS[tr.split()[pos_word]]:
                                is_ambiguous = True
                                break
                        
                        if is_ambiguous:
                            if verbose:
                                print('token check_word: ', target_sent[id_sense])
                            temp.append(target_sent[id_sense])
                        else:
                            temp.append('no_instance')
                            
                    elif len(input_sent_tokens) <= 4 and len(trns_sent_tokens) <= 4:   
                        
                        is_ambiguous = False
                        tag_tr = tags_tr[pos_word]
                        
                        tag_cntr_or = Counter([tag[1] for tag in tags_or[ant_n_words_or:(ant_n_words_or+n_words_or)]])
                        tag_cntr_tr = Counter([tag[1] for tag in tags_tr[ant_n_words_tr:(ant_n_words_tr+n_words_tr)]])

                        for id_sense, tag_or in zip(sent_to_id[ant_n_words_or:(ant_n_words_or+n_words_or)], tags_or[ant_n_words_or:(ant_n_words_or+n_words_or)]):
                            if target_sent[id_sense] != 'no_instance' and tag_or[1] == tag_tr[1] and tag_cntr_tr[tag_tr[1]] == 1 and tag_cntr_or[tag_or[1]] == 1: 
                                is_ambiguous = True
                                temp_id = target_sent[id_sense]
                                if verbose:
                                    print('sentido encontrado: ', temp_id)
                                break
                                
                        if is_ambiguous:
                            temp.append(temp_id)
                        else:
                            temp.append('no_instance')

                    else:
                        temp.append('no_instance')
                else:
                    temp.append('no_instance')
            
            tagged_tr.append([temp, tr, input_sent])
    
    return tagged_tr

In [87]:
def get_verbs(sent):
    temp_cont = []
    for ix, word in enumerate(sent.split()):
        if nlp_pt(word)[0].lemma_ in PT_VERBS:
            temp_cont.append(ix)
    return temp_cont

In [208]:
def count_instances_with_verbs(tagged_tr):
    cont_d = []
    cont_n = []
    for ix_instance, instance in enumerate(tagged_tr):
        ambiguous=False
        for ix_word, word in enumerate(instance[1].split()):
            if instance[0][ix_word] != 'no_instance' and nlp_pt(word)[0].lemma_ in PT_VERBS:
                cont_d.append(ix_instance)
                ambiguous=True
                break
        if not ambiguous:
            cont_n.append(ix_instance)
                
    return cont_d, cont_n

In [276]:
def filter_index(metrics, sims, porc, p=''):
    
    filtered_ids = []
    if p == 'm':
        threshold_metric = np.mean(metrics)
    else:
        threshold_metric = np.mean(metrics)
        
    threshold_sim = np.mean(sims) + np.std(sims)
    ordered_indexes = np.argsort(-metrics)
    filtered_ids = ordered_indexes[:int(len(ordered_indexes) * porc)]
    #for ix, (metric, sim) in enumerate(zip(metrics, sims)):
    #    if metric > threshold_metric:
    #        filtered_ids.append(ix)
    #    elif sim > threshold_sim:
    #        filtered_ids.append(ix)
                
    print(f'{p:15s}total ids: {len(metrics)}\t|\t After filter: {len(filtered_ids)}')
    return filtered_ids

In [90]:
def get_pos_tags(orig_sentences, trns_sentences): 
    tags_tr = []
    for sent in (trns_sentences):
        if len(sent):
            tag_tr = []
            temp = nlp_pt(str(sent).strip())
            for token in temp:
                tag_tr.append([token.text, token.pos_])
            tags_tr.append(tag_tr)
        else:
            tags_tr.append(['invalid', 'invalid'])
    
    tags_or = []
    for sent in (orig_sentences):
        if len(sent):
            tag_or = []
            temp = nlp_en(str(sent).strip())
            for token in temp:
                tag_or.append([token.text, token.pos_])
            tags_or.append(tag_or)
        else:
            tags_or.append(['invalid', 'invalid'])
        
    return tags_or, tags_tr

In [173]:
def filter_index_by_porc(metrics, sims, porc, p=''):
    
    ordered_indexes = np.argsort(-metrics)
    filtered_ids = ordered_indexes[:int(len(ordered_indexes) * porc)]
                
    print(f'{p:15s}total ids: {len(metrics)}\t|\t After filter: {len(filtered_ids)}')
    return filtered_ids

In [277]:
def filter_index_by_mean(metrics, sims, p=''):
    
    filtered_ids = []
    if p == 'm':
        threshold_metric = np.mean(metrics) - np.std(metrics)
    else:
        threshold_metric = np.mean(metrics)# + np.std(metrics)
    threshold_sim = np.mean(sims)# + np.std(sims)
    #ordered_indexes = np.argsort(-metrics)
    #filtered_ids = ordered_indexes[:int(len(ordered_indexes) * porc)]
    for ix, (metric, sim) in enumerate(zip(metrics, sims)):
        if metric > threshold_metric:
            filtered_ids.append(ix)
        #elif sim > threshold_sim:
        #    filtered_ids.append(ix)
                
    print(f'{p:15s}total ids: {len(metrics)}\t|\t After filter: {len(filtered_ids)}')
    return filtered_ids

# Saving data

In [91]:
wordnet_senses = {}
with open(Path.cwd() / 'data/wordnet/index.sense') as f:
    for line in f.read().split('\n'):
        if len(line) > 1:
            wordnet_senses[line.split()[0]] = line.split()[1]
f.close()

In [92]:
def load_senses(path):
    d_senses = {}
    with open(Path.cwd() / path, 'r') as f:
        for line in f.read().split('\n'):
            if len(line):
                temp = line.split()
                d_senses[temp[0]] = temp[1:]
    return d_senses
                
d_senses_semcor = load_senses('data/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt')
d_senses_omsti = load_senses('data/WSD_Evaluation_Framework/Training_Corpora/SemCor+OMSTI/semcor+omsti.gold.key.txt')
d_senses_dev = load_senses('data/WSD_Evaluation_Framework/Evaluation_Datasets/ALL/all.gold.key.txt')

In [99]:
pos_or_semcor = np.load('data/serialize/pos_or_semcor.npy')
pos_tr_semcor = np.load('data/serialize/pos_tr_semcor.npy')

pos_or_omsti = np.load('data/serialize/pos_or_omsti.npy')
pos_tr_omsti = np.load('data/serialize/pos_tr_omsti.npy')

pos_or_dev = np.load('data/serialize/pos_or_dev.npy')
pos_tr_dev = np.load('data/serialize/pos_tr_dev.npy')

In [98]:
#pos_or_dev, pos_tr_dev = get_pos_tags(input_sent_dev, tr_dev)

In [103]:
verbose=False
tagged_tr_semcor = tagged_translations(list(range(0, len(id_semcor))), id_semcor, al_semcor, tr_semcor, input_sent_semcor, sent_to_id_semcor, target_sent_semcor, pos_or_semcor, pos_tr_semcor)
tagged_tr_omsti = tagged_translations(list(range(0, len(id_omsti))), id_omsti, al_omsti, tr_omsti, input_sent_omsti, sent_to_id_omsti, target_sent_omsti, pos_or_omsti, pos_tr_omsti)
tagged_tr_dev = tagged_translations(list(range(0, len(id_dev))), id_dev, al_dev, tr_dev, input_sent_dev, sent_to_id_dev, target_sent_dev, pos_or_dev, pos_tr_dev)

In [106]:
tagged_all = np.concatenate((tagged_tr_semcor, tagged_tr_omsti, tagged_tr_dev), axis=0)

In [107]:
ix_disamb_all, ix_normal_all = count_instances_with_verbs(tagged_all)

In [364]:
ann_semcor = annotate_sentences(tagged_tr_semcor, d_senses_semcor, lemma=False, sel_verbs=True)
ann_omsti = annotate_sentences(tagged_tr_omsti, d_senses_omsti, lemma=False, sel_verbs=True)
ann_dev = annotate_sentences(tagged_tr_dev, d_senses_dev, lemma=False, sel_verbs=True)

KeyboardInterrupt: 

In [223]:
tagged_sense_all = np.concatenate((ann_semcor[0], ann_omsti[0], ann_dev[0]), axis=0)
input_sense_all = np.concatenate((ann_semcor[1], ann_omsti[1], ann_dev[1]), axis=0)
orig_sent_all = np.concatenate((ann_semcor[2], ann_omsti[2], ann_dev[2]), axis=0)

In [314]:
# Checkpoint
#np.save('data/serialize/f1_all.npy', f1_all)

#np.save('data/serialize/meteor_semcor.npy', meteor_semcor)
#np.save('data/serialize/meteor_omsti.npy', meteor_omsti)
#np.save('data/serialize/meteor_dev.npy', meteor_dev)

#np.save('data/serialize/sim_semcor.npy', sim_semcor)
#np.save('data/serialize/sim_omsti.npy', sim_omsti)
#np.save('data/serialize/sim_dev.npy', sim_dev)

#np.save('data/serialize/ann_semcor.npy', ann_semcor)
#np.save('data/serialize/ann_omsti.npy', ann_omsti)
#np.save('data/serialize/ann_dev.npy', ann_dev)

#np.save('data/serialize/tagged_tr_semcor.npy', tagged_tr_semcor)
#np.save('data/serialize/tagged_tr_omsti.npy', tagged_tr_omsti)
#np.save('data/serialize/tagged_tr_dev.npy', tagged_tr_dev)

#np.save('data/serialize/ix_disamb_all.npy', ix_disamb_all)
#np.save('data/serialize/ix_normal_all.npy', ix_normal_all)

### Annotate all

In [219]:
quartile_f1 = np.percentile(np.sort(f1_all), 75)
quartile_meteor = np.percentile(np.sort(meteor_all), 75)

temp_ix_f1 = np.where(f1_all > quartile_f1)[0]
temp_ix_meteor = np.where(meteor_all > quartile_meteor)[0]

temp = np.intersect1d(temp_ix_f1, temp_ix_meteor)
temp_ix_inters = np.intersect1d(temp, ix_disamb_all)
print('> Q3 METEOR :', len(temp_ix_meteor))
print('> Q3 F1 :', len(temp_ix_f1))
print('INTERSECCION METEOR Y F1 :', len(temp))
print('ORACIONES AMBIGUAS EN INTERSECCION', len(temp_ix_inters))
print('ORACIONES AMBIGUAS EN TODO EL CORPUS', len(ix_disamb_all))

> Q3 METEOR : 19392
> Q3 F1 : 19392
INTERSECCION METEOR Y F1 : 14178
ORACIONES AMBIGUAS EN INTERSECCION 4107
ORACIONES AMBIGUAS EN TODO EL CORPUS 18160


In [242]:
quartile_f1 = np.percentile(np.sort(f1_all[ix_disamb_all]), 75)
quartile_meteor = np.percentile(np.sort(meteor_all[ix_disamb_all]), 75)

temp_ix_f1 = np.where(f1_all > quartile_f1)[0]
temp_ix_meteor = np.where(meteor_all > quartile_meteor)[0]

temp = np.intersect1d(temp_ix_f1, temp_ix_meteor)
ix_dev = np.intersect1d(temp, ix_disamb_all)
print('> Q3 METEOR :', len(temp_ix_meteor))
print('> Q3 F1 :', len(temp_ix_f1))
print('INTERSECCION METEOR Y F1 :', len(temp))
print('ORACIONES AMBIGUAS EN INTERSECCION', len(temp_ix_inters))
print('ORACIONES AMBIGUAS EN TODO EL CORPUS', len(ix_disamb_all))

> Q3 METEOR : 16358
> Q3 F1 : 15506
INTERSECCION METEOR Y F1 : 13139
ORACIONES AMBIGUAS EN INTERSECCION 3784
ORACIONES AMBIGUAS EN TODO EL CORPUS 18160


In [243]:
ix_train = [i  for i in range(len(f1_all)) if i not in ix_dev]

In [244]:
dev_path = Path.cwd() / 'data/disambiguation/dev'
save_tagged_sentence(dev_path, 'dev', tagged_sense_all.take(ix_dev), 
                                     input_sense_all.take(ix_dev),
                                     orig_sent_all.take(ix_dev),
                                     mode='w')

In [281]:
f1_train = f1_all.take(ix_train)
meteor_train = meteor_all.take(ix_train)
sim_train = sim_all.take(ix_train)

tagged_sense_train = tagged_sense_all.take(ix_train)
input_sense_train = input_sense_all.take(ix_train)
orig_sent_train = orig_sent_all.take(ix_train)

In [398]:
len(f1_train)

73784

In [353]:
def sampling_random(len_normal, len_disamb, len_verb, ix_normal, ix_disamb, ix_verb, seed):
    np.random.seed(seed)
    temp_normal = np.random.choice(len(ix_normal), len_normal, replace=False)
    np.random.seed(seed)
    temp_disamb = np.random.choice(len(ix_disamb), len_disamb, replace=False)
    np.random.seed(seed)
    temp_verb = np.random.choice(len(ix_verb), len_verb, replace=False)
    return np.concatenate((np.array(ix_normal).take(temp_normal), 
                           np.array(ix_verb).take(temp_verb)), axis=0)

In [304]:
len(ix_disamb_train), len(ix_verbs_train)

(7308, 15915)

In [338]:
len(ix_disamb_train), len(ix_verbs_train)

(0, 14376)

In [336]:
input_sense_aux = []
tagged_sense_aux = []
for ix in range(len(input_sense_train)):
    s_in = []
    s_out = []
    if '_tag' in input_sense_train[ix]:
        for ix_word, (word_in, word_out) in enumerate(zip(input_sense_train[ix].split(), tagged_sense_train[ix].split())):
            if '_tag' in word:
                if word_in.replace('_tag', '') in PT_VERBS:
                    s_in.append(word_in)
                    s_out.append(word_out)
                else:
                    s_in.append(word_in.replace('_tag', ''))
                    s_out.append(word_in.replace('_tag', ''))
            else:
                s_in.append(word_in)
                s_out.append(word_out)
        input_sense_aux.append(' '.join(s_in))
        tagged_sense_aux.append(' '.join(s_out)) 
    else:
        input_sense_aux.append(input_sense_train[ix])
        tagged_sense_aux.append(tagged_sense_train[ix]) 

In [342]:
input_sense_aux = np.array(input_sense_aux)
tagged_sense_aux = np.array(tagged_sense_aux)

In [337]:
ix_verbs_train = []
ix_disamb_train = []
for ix in range(len(tagged_sense_aux)):
    if '_tag' in input_sense_aux[ix]:
        flag = False
        for word in input_sense_aux[ix].split():
            if word.replace('_tag', '') in PT_VERBS:
                flag = True
                break
        if flag:
            ix_verbs_train.append(ix)
        else:
            ix_disamb_train.append(ix)
        
ix_normal_train = []
for ix in range(len(tagged_sense_aux)):
    if '_tag' not in input_sense_aux[ix]:
        ix_normal_train.append(ix)

In [369]:
len(f1)

NameError: name 'f1' is not defined

In [371]:
def gen_temp(f1, meteor, sim, ix_normal, ix_disamb, ix_verbs, tagged_sense_aux, input_sense_aux, orig_sent_train):
    for ter in [0.4]:
        print(f'particion {100 * ter}. ')
        filtered_ids_f = filter_index_by_porc(f1, sim, ter, p='f1')
        filtered_ids_m = filter_index_by_porc(meteor, sim, ter, p='m')

        # Numero de instancias ambiguas
        n_amb_f = np.intersect1d(filtered_ids_f, ix_disamb)
        n_ver_f = np.intersect1d(filtered_ids_f, ix_verbs)

        n_amb_m = np.intersect1d(filtered_ids_m, ix_disamb)
        n_ver_m = np.intersect1d(filtered_ids_m, ix_verbs)
    return filtered_ids_f, filtered_ids_m

In [373]:
filtered_ids_f, filtered_ids_m = gen_temp(f1_train, meteor_train, sim_train,
                    ix_normal_train, ix_disamb_train, ix_verbs_train,
                   tagged_sense_aux, input_sense_aux, orig_sent_train)

particion 40.0. 
f1             total ids: 73784	|	 After filter: 29513
m              total ids: 73784	|	 After filter: 29513


In [377]:
len(orig_sent_train.take(filtered_ids_f))

29513

In [381]:
len(orig_sent_train.take(filtered_ids_m))

29513

In [382]:
ix_f = (list(set(orig_sent_train.take(filtered_ids_f))))

In [383]:
ix_m = (list(set(orig_sent_train.take(filtered_ids_m))))

In [390]:
len(or_all)

77568

In [ ]:
trns_f = []
for i in range(len(ix_f)):
    for ix in range(len(or_all)):
        if or_all[i] == ix_f[i]:
            trns_f.append(tr_all[i])

In [ ]:
ix_f[0]

In [ ]:
trns_f[0]

In [395]:
i

'meetings with donors that were launched during the final quarter of 2008 with the aim of providing a detailed explanation of the revised technical assistance strategy of the executive directorate and of discussing areas in which they might be able to provide technical assistance , will continue during 2009 .'

In [393]:
orig_sent_train[0]

'how long has it been since you reviewed the objectives of your benefit and service program ?'

In [399]:
def generate_partitions(f1, meteor, sim, ix_normal, ix_disamb, ix_verbs, tagged_sense_aux, input_sense_aux, orig_sent_train):
    for ter in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
        print(f'particion {100 * ter}. ')
        filtered_ids_f = filter_index_by_porc(f1, sim, ter, p='f1')
        filtered_ids_m = filter_index_by_porc(meteor, sim, ter, p='m')

        # Numero de instancias ambiguas
        n_amb_f = np.intersect1d(filtered_ids_f, ix_disamb)
        n_ver_f = np.intersect1d(filtered_ids_f, ix_verbs)

        n_amb_m = np.intersect1d(filtered_ids_m, ix_disamb)
        n_ver_m = np.intersect1d(filtered_ids_m, ix_verbs)
        print(f'{len(n_ver_f)} oraciones ambiguas en f1, {len(n_ver_m)} oraciones ambiguas en meteor')

        temp_path = Path.cwd() / f'data/disambiguation/all_{int(ter * 100)}'
        save_tagged_sentence(temp_path, 'all_f1_lemma', tagged_sense_aux.take(filtered_ids_f), input_sense_aux.take(filtered_ids_f), orig_sent_train.take(filtered_ids_f))

        ix_random = sampling_random(len(filtered_ids_f) - (len(n_amb_f) + len(n_ver_f)), 
                                    len(n_amb_f),
                                    len(n_ver_f),
                                    ix_normal, 
                                    ix_disamb, 
                                    ix_verbs,
                                    SEED+30)
        save_tagged_sentence(temp_path, 'all_f1_random_lemma', tagged_sense_aux.take(ix_random), input_sense_aux.take(ix_random), orig_sent_train.take(ix_random))
        np.save(temp_path / 'r_b.npy', f1.take(filtered_ids_f))
        np.save(temp_path / 'meteor.npy', meteor.take(filtered_ids_m))
        save_tagged_sentence(temp_path, 'all_meteor_lemma', tagged_sense_aux.take(filtered_ids_m), input_sense_aux.take(filtered_ids_m), orig_sent_train.take(filtered_ids_m))

        ix_random = sampling_random(len(filtered_ids_m) - (len(n_amb_m) + len(n_ver_m)), 
                                    len(n_amb_m),
                                    len(n_ver_m),
                                    ix_normal, 
                                    ix_disamb, 
                                    ix_verbs,
                                    SEED+30)
        save_tagged_sentence(temp_path, 'all_meteor_random_lemma', tagged_sense_aux.take(ix_random), input_sense_aux.take(ix_random), orig_sent_train.take(ix_random))

        print()

    print('all eq partition')

    filtered_ids_f = filter_index_by_mean(f1, sim, p='f1')
    filtered_ids_m = filter_index_by_mean(meteor, sim, p='m')

    # Numero de instancias ambiguas
    n_amb_f = np.intersect1d(filtered_ids_f, ix_disamb)
    n_ver_f = np.intersect1d(filtered_ids_f, ix_verbs)

    n_amb_m = np.intersect1d(filtered_ids_m, ix_disamb)
    n_ver_m = np.intersect1d(filtered_ids_m, ix_verbs)
    print(f'{len(n_ver_f)} oraciones ambiguas en f1, {len(n_ver_m)} oraciones ambiguas en meteor')

    temp_path = Path.cwd() / f'data/disambiguation/all_eq'
    print('Saving rouge and bleu')
    save_tagged_sentence(temp_path, 'all_f1_lemma', tagged_sense_aux.take(filtered_ids_f), input_sense_aux.take(filtered_ids_f), orig_sent_train.take(filtered_ids_f))

    ix_random = sampling_random(len(filtered_ids_f) - (len(n_amb_f) + len(n_ver_f)), 
                                len(n_amb_f),
                                len(n_ver_f),
                                ix_normal, 
                                ix_disamb, 
                                ix_verbs,
                                SEED+30)
    save_tagged_sentence(temp_path, 'all_f1_random_lemma', tagged_sense_aux.take(ix_random), input_sense_aux.take(ix_random), orig_sent_train.take(ix_random))

    print('Saving meteor')
    save_tagged_sentence(temp_path, 'all_meteor_lemma', tagged_sense_aux.take(filtered_ids_m), input_sense_aux.take(filtered_ids_m), orig_sent_train.take(filtered_ids_m))

    ix_random = sampling_random(len(filtered_ids_m) - (len(n_amb_m) + len(n_ver_m)), 
                                len(n_amb_m),
                                len(n_ver_m),
                                ix_normal, 
                                ix_disamb, 
                                ix_verbs,
                                SEED+30)
    save_tagged_sentence(temp_path, 'all_meteor_random_lemma', tagged_sense_aux.take(ix_random), input_sense_aux.take(ix_random), orig_sent_train.take(ix_random))

In [361]:
def sampling_random(len_normal, len_disamb, len_verb, ix_normal, ix_disamb, ix_verb, seed):
    np.random.seed(seed)
    temp_normal = np.random.choice(len(ix_normal), len_normal, replace=False)
    np.random.seed(seed)
    temp_disamb = np.random.choice(len(ix_disamb), len_disamb, replace=False)
    np.random.seed(seed)
    temp_verb = np.random.choice(len(ix_verb), len_verb, replace=False)
    return np.concatenate((np.array(ix_normal).take(temp_normal), 
                           np.array(ix_verb).take(temp_verb)), axis=0)

generate_partitions(f1_train, meteor_train, sim_train,
                    ix_normal_train, ix_disamb_train, ix_verbs_train,
                   tagged_sense_aux, input_sense_aux, orig_sent_train)

particion 20.0. 
f1             total ids: 73784	|	 After filter: 14756
m              total ids: 73784	|	 After filter: 14756
1657 oraciones ambiguas en f1, 1250 oraciones ambiguas en meteor

particion 30.0. 
f1             total ids: 73784	|	 After filter: 22135
m              total ids: 73784	|	 After filter: 22135
3801 oraciones ambiguas en f1, 3283 oraciones ambiguas en meteor

particion 40.0. 
f1             total ids: 73784	|	 After filter: 29513
m              total ids: 73784	|	 After filter: 29513
5695 oraciones ambiguas en f1, 5143 oraciones ambiguas en meteor

particion 50.0. 
f1             total ids: 73784	|	 After filter: 36892
m              total ids: 73784	|	 After filter: 36892
7394 oraciones ambiguas en f1, 6880 oraciones ambiguas en meteor

particion 60.0. 
f1             total ids: 73784	|	 After filter: 44270
m              total ids: 73784	|	 After filter: 44270
8919 oraciones ambiguas en f1, 8540 oraciones ambiguas en meteor

particion 70.0. 
f1             tot

In [362]:
tagged_sense_aux.take(ix_verbs_train[0])

'você tem permitir que ele se tornasse um programa de doação, ao invés de uma, que 02630189 a meta de melhoria do moral dos funcionários e , consequentemente , o aumento da produtividade ?'

In [400]:
def sampling_random(len_normal, len_disamb, len_verb, ix_normal, ix_disamb, ix_verb, seed):
    np.random.seed(seed)
    temp_normal = np.random.choice(len(ix_normal), len_normal, replace=False)
    np.random.seed(seed)
    temp_disamb = np.random.choice(len(ix_disamb), len_disamb, replace=False)
    np.random.seed(seed)
    temp_verb = np.random.choice(len(ix_verb), len_verb, replace=False)
    return np.array(ix_verb).take(temp_verb)

generate_partitions(f1_train.take(ix_verbs_train), meteor_train.take(ix_verbs_train), sim_train.take(ix_verbs_train),
                    [], [], list(range(len(ix_verbs_train))),
                   tagged_sense_aux.take(ix_verbs_train), input_sense_aux.take(ix_verbs_train), orig_sent_train.take(ix_verbs_train))

particion 20.0. 
f1             total ids: 14376	|	 After filter: 2875
m              total ids: 14376	|	 After filter: 2875
2875 oraciones ambiguas en f1, 2875 oraciones ambiguas en meteor

particion 30.0. 
f1             total ids: 14376	|	 After filter: 4312
m              total ids: 14376	|	 After filter: 4312
4312 oraciones ambiguas en f1, 4312 oraciones ambiguas en meteor

particion 40.0. 
f1             total ids: 14376	|	 After filter: 5750
m              total ids: 14376	|	 After filter: 5750
5750 oraciones ambiguas en f1, 5750 oraciones ambiguas en meteor

particion 50.0. 
f1             total ids: 14376	|	 After filter: 7188
m              total ids: 14376	|	 After filter: 7188
7188 oraciones ambiguas en f1, 7188 oraciones ambiguas en meteor

particion 60.0. 
f1             total ids: 14376	|	 After filter: 8625
m              total ids: 14376	|	 After filter: 8625
8625 oraciones ambiguas en f1, 8625 oraciones ambiguas en meteor

particion 70.0. 
f1             total ids: 14

In [357]:
15000 * 0.2

3000.0

f1             total ids: 73784	|	 After filter: 40870
m              total ids: 73784	|	 After filter: 68757
8241 oraciones ambiguas en f1, 13488 oraciones ambiguas en meteor
Saving rouge and bleu
Saving meteor


In [309]:
len(filtered_ids_m) - (len(n_amb_m) + len(n_ver_m))

50720

In [310]:
len(ix_normal_train)

50561

In [274]:
from src.data import prepare_data

dir_files = 'data/disambiguation/'
dir_test = os.path.join(dir_files, 'test')
dir_dev = os.path.join(dir_files, 'dev')

for name_dir in ['all_20', 'all_30', 'all_40', 'all_50', 'all_60', 'all_eq', 'all_less']:
    dir_train = os.path.join(dir_files, name_dir)
    for name_file in ['all_f1_lemma', 'all_f1_random_lemma', 'all_meteor_lemma', 'all_meteor_random_lemma']:
        _, _, pairs_train, pairs_dev, pairs_test, _= prepare_data(name_file, 'verbs_selected_lemma',
                                                                       max_length=100, dir_train=dir_train, 
                                                                       dir_dev=dir_dev, dir_test=dir_test)
        
        pd.DataFrame(pairs_train).to_csv(os.path.join(dir_train, name_file + '.tsv'), header=False, index=False, sep='\t')

pd.DataFrame(pairs_dev).to_csv(os.path.join(dir_dev, 'verbs_selected_lemma.tsv'), header=False, index=False, sep='\t')
pd.DataFrame(pairs_test).to_csv(os.path.join(dir_test, 'verbs_selected_lemma.tsv'), header=False, index=False, sep='\t')

TypeError: prepare_data() got an unexpected keyword argument 'dir_dev'